In [ ]:
import os
import re
import time
import nltk
import string
import tensorlayer as tl
from utils import *
from collections import Counter
import tensorflow as tf

from tensorlayer.layers import *
from tensorlayer.prepro import *
from tensorlayer.cost import *


In [ ]:
cwd = os.getcwd()
print("1 : cwd " + cwd)
img_dir = os.path.join(cwd, 'CUB\\images')
print("1 : img_dir " + img_dir)

caption_dir = os.path.join(cwd, 'birds\\text_c10')
print("1 : caption_dir " + caption_dir)

VOC_FIR = cwd + '\\vocab.txt'
print("1 : VOC_FIR " + VOC_FIR)


In [ ]:
## load captions
caption_sub_dir = load_folder_list( caption_dir )
caption_sub_dir
captions_dict = {}
processed_capts = []

In [ ]:


batch_size = 16
z_dim = 1024          #### changed 1024 <------ 512 
image_size = 256    
c_dim = 3          
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.2)



import pickle
import numpy as np

with open("_vocab.pickle", 'rb') as f:
     vocab = pickle.load(f)
with open("_image_train.pickle", 'rb') as f:
     images_train,_ = pickle.load(f)
with open("_image_test.pickle", 'rb') as f:
     images_test,_ = pickle.load(f)
with open("_n.pickle", 'rb') as f:
     n_captions_train, n_captions_test, n_captions_per_image, n_images_train, n_images_test = pickle.load(f)
with open("_caption.pickle", 'rb') as f:
     captions_ids_train, captions_ids_test = pickle.load(f)

        
print(len(images_train))
print(len(images_train[0]))
print(len(images_train[1]))
print(len(images_train[2]))
print(len(images_train[3]))
print(len(images_train[0][0]))
print(len(images_train[0][0][0]))

print(type(images_train))
print(type(images_train[0]))
print(type(images_train[0][0]))
print(type(images_train[0][0][0]))
        
        
        

images_train = np.array(images_train).astype(np.float32)
images_test = np.array(images_test)
ni = int(np.ceil(np.sqrt(batch_size)))


print(len(images_train))
print(len(images_train[0]))
print(len(images_train[0][0]))
print(len(images_train[0][0][0]))

print(type(images_train))
print(type(images_train[0]))
print(type(images_train[0][0]))
print(type(images_train[0][0][0]))
        




if not os.path.exists("samples/step1_gan-cls"):
    os.makedirs("samples/step1_gan-cls")
if not os.path.exists("samples/step_pretrain_encoder"):
    os.makedirs("samples/step_pretrain_encoder")        
if not os.path.exists("checkpoint"):
    os.makedirs("checkpoint")
save_dir = "checkpoint"


w_init = tf.random_normal_initializer(stddev=0.02)
gamma_init = tf.random_normal_initializer(1., 0.02)
df_dim = 64
inputs = []
## for text-to-image mapping ===================================================
t_dim = 256         # text feature dimension # changed 256 <---- 128
rnn_hidden_size = t_dim
vocab_size = 8000
word_embedding_size = 256
keep_prob = 1.0




    

def cnn_encoder(inputs, is_train=True, reuse=False, name='cnnftxt', return_h3=False):
    """ 64x64 --> t_dim, for text-image mapping """
    w_init = tf.random_normal_initializer(stddev=0.02)
    gamma_init = tf.random_normal_initializer(1., 0.02)
    df_dim = 256

    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        tl.layers.set_name_reuse(tf.AUTO_REUSE)

        net_in = InputLayer(inputs, name='/in')
        net_h0 = Conv2d(net_in, df_dim, (4, 4), (2, 2), act=lambda x: tl.act.lrelu(x, 0.2),
                padding='SAME', W_init=w_init, name='cnnf/h0/conv2d')

        net_h1 = Conv2d(net_h0, df_dim*2, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='cnnf/h1/conv2d')
        net_h1 = BatchNormLayer(net_h1, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='cnnf/h1/batch_norm')

        # if name != 'cnn': # debug for training image encoder in step 2
        #     net_h1 = DropoutLayer(net_h1, keep=0.8, is_fix=True, name='p/h1/drop')

        net_h2 = Conv2d(net_h1, df_dim*4, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='cnnf/h2/conv2d')
        net_h2 = BatchNormLayer(net_h2, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='cnnf/h2/batch_norm')

        # if name != 'cnn': # debug for training image encoder in step 2
        #     net_h2 = DropoutLayer(net_h2, keep=0.8, is_fix=True, name='p/h2/drop')

        net_h3 = Conv2d(net_h2, df_dim*8, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='cnnf/h3/conv2d')
        net_h3 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='cnnf/h3/batch_norm')

        # if name != 'cnn': # debug for training image encoder in step 2
        #     net_h3 = DropoutLayer(net_h3, keep=0.8, is_fix=True, name='p/h3/drop')

        net_h4 = FlattenLayer(net_h3, name='cnnf/h4/flatten')
        net_h4 = DenseLayer(net_h4, n_units= (z_dim if name == 'z_encoder' else t_dim),
                act=tf.identity,
                W_init = w_init, b_init = None, name='cnnf/h4/embed')
    if return_h3:
        return net_h4, net_h3
    else:
        return net_h4
    

In [ ]:

def rnn_embed(input_seqs, is_train=True, reuse=False, return_embed=False):
    """ txt --> t_dim """
    w_init = tf.random_normal_initializer(stddev=0.02)
    if tf.__version__ <= '0.12.1':
        LSTMCell = tf.nn.rnn_cell.LSTMCell
    else:
        LSTMCell = tf.contrib.rnn.BasicLSTMCell
    with tf.variable_scope("rnnftxt", reuse=tf.AUTO_REUSE):
        tl.layers.set_name_reuse(True)
        network = EmbeddingInputlayer(
                     inputs = input_seqs,
                     vocabulary_size = vocab_size,
                     embedding_size = word_embedding_size,
                     E_init = w_init,
                     name = 'rnn/wordembed')
        network = DynamicRNNLayer(network,
                     cell_fn = LSTMCell,
                     cell_init_args = {'state_is_tuple' : True, 'reuse': reuse},  # for TF1.1, TF1.2 dont need to set reuse
                     n_hidden = rnn_hidden_size,
                     dropout = (keep_prob if is_train else None),
                     initializer = w_init,
                     sequence_length = tl.layers.retrieve_seq_length_op2(input_seqs),
                     return_last = True,
                     name = 'rnn/dynamic')
        return network


In [ ]:

def discriminator_txt2img_resnet(input_images, t_txt=None, is_train=True, reuse=False):
    """ 64x64 + (txt) --> real/fake """
    # https://github.com/hanzhanggit/StackGAN/blob/master/stageI/model.py
    # Discriminator with ResNet : line 197 https://github.com/reedscot/icml2016/blob/master/main_cls.lua
    w_init = tf.random_normal_initializer(stddev=0.02)
    gamma_init=tf.random_normal_initializer(1., 0.02)
    df_dim = 64  # 64 for flower, 196 for MSCOCO
    s = 256 # output image size [64]
    s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)
    

    with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
        tl.layers.set_name_reuse(True)
        net_in = InputLayer(input_images, name='d_input/images')
        print("1   ",net_in.outputs.shape )
    
        net_h0 = Conv2d(net_in, df_dim, (4, 4),(2,2), act=lambda x: tl.act.lrelu(x, 0.2),
                padding='SAME', W_init=w_init, name='d_h0/conv2d')
        
        
        print("2   ",net_h0.outputs.shape )
        
        
        net_h1 = Conv2d(net_h0, df_dim*2, (4, 4), (2,2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h1/conv2d')
        net_h1 = BatchNormLayer(net_h1, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h1/batchnorm')
        
        
        print("3   ",net_h1.outputs.shape )
        
        
        net_h2 = Conv2d(net_h1, df_dim*4, (4, 4), (2,2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h2/conv2d')
        net_h2 = BatchNormLayer(net_h2, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h2/batchnorm')
        
        
        print("3.1   ",net_h2.outputs.shape )
        net_h3 = Conv2d(net_h2, df_dim*8, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h3/conv2d') # new co
        
        net_h3_1 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h3/batchnorm')
        
        
        print("4.1  ",net_h3_1.outputs.shape )
        
        
        net_h3 = Conv2d(net_h3_1, df_dim*16, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h311/conv2dd')
        net_h3 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h311/batchnormmm')
        print("5  ",net_h3.outputs.shape )
        
        net_h3 = Conv2d(net_h3, df_dim*32, (4, 4), (2, 2), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h322/conv2ddd')
        net_h3 = BatchNormLayer(net_h3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h322/batchnormmmm')
        print("6  ",net_h3.outputs.shape )
        
        
        
        net1 = Conv2d(net_h3, df_dim*16, (1, 1), (1, 1), act=None,
                padding='VALID', W_init=w_init, b_init=None, name='d_h4_res0/0conv2d')
        net1 = BatchNormLayer(net1, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h4_res0/0batchnorm')
        
        print("6.1  ",net1.outputs.shape )
        net2 = Conv2d(net1, df_dim*8, (1, 1), (1, 1), act=None,
                padding='VALID', W_init=w_init, b_init=None, name='d_h4_res1/1conv2d')
        net2 = BatchNormLayer(net2, #act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h4_res1/1batchnorm')# code old add
        print("6.2  ",net2.outputs.shape )
        
        print("6.3  ",net1.outputs.shape )
        net3 = Conv2d(net2, df_dim*2, (1, 1), (1, 1), act=None,
                padding='VALID', W_init=w_init, b_init=None, name='d_h4_res10/10conv2d')
        net3 = BatchNormLayer(net3, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h4_res10/10batchnorm')
        print("6.3  ",net2.outputs.shape )
    
    
                
            
        net = Conv2d(net3, df_dim*2, (3, 3), (1, 1), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h4_res/conv2d2')
        net = BatchNormLayer(net, act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h4_res/batchnorm2')
        print("8  ",net.outputs.shape )
        
        
        net = Conv2d(net, df_dim*8, (3, 3), (1, 1), act=None,
                padding='SAME', W_init=w_init, b_init=None, name='d_h4_res/conv2d3')
        net = BatchNormLayer(net, #act=lambda x: tl.act.lrelu(x, 0.2),
                is_train=is_train, gamma_init=gamma_init, name='d_h4_res/batchnorm3')
        
        
        
        print("9  ",net.outputs.shape )
        net_h4 = ElementwiseLayer(layer=[net2, net], combine_fn=tf.add, name='d_h4/add')
        net_h4.outputs = tl.act.lrelu(net_h4.outputs, 0.2)
        print("10 ",net_h4.outputs.shape )

        if t_txt is not None:
            net_txt = InputLayer(t_txt, name='d_input_txt')
            net_txt = DenseLayer(net_txt, n_units=t_dim,
                   act=lambda x: tl.act.lrelu(x, 0.2),
                   W_init=w_init, name='d_reduce_txt/dense')
            print("DA ",net_txt.outputs.shape )
            net_txt = ExpandDimsLayer(net_txt, 1, name='d_txt/expanddim1')
            print("DA  da ",net_txt.outputs.shape )
            net_txt = ExpandDimsLayer(net_txt, 1, name='d_txt/expanddim2')
            print("DA da da ",net_txt.outputs.shape )
            net_txt = TileLayer(net_txt, [1, 4, 4, 1], name='d_txt/tile')
            print("DA da da da",net_txt.outputs.shape )
            print("DA da da da",net_h4.outputs.shape )
            
            
            net_h4_concat = ConcatLayer([net_h4, net_txt], concat_dim=3, name='d_h3_concat')
            # 243 (ndf*8 + 128 or 256) x 4 x 4
            print("DA 111 ",net_h4_concat.outputs.shape )
            net_ho = Conv2d(net_h4_concat, df_dim*8, (1, 1), (1, 1),
                    padding='VALID', W_init=w_init, b_init=None, name='d_h3/conv2d_2')
            net_ho = BatchNormLayer(net_ho, act=lambda x: tl.act.lrelu(x, 0.2),
                    is_train=is_train, gamma_init=gamma_init, name='d_h3/batch_norm_2')
            print("DA 111 2222 ",net_ho.outputs.shape )
            
           

        #net_ho = Conv2d(net_h4, 1, (s16, s16), (s16, s16), padding='VALID', W_init=w_init, name='d_hop/pconv2d')

        net_ho = FlattenLayer(net_ho, name='d_hop/pflatten')
        logits = net_ho.outputs
        net_ho.outputs = tf.nn.sigmoid(net_ho.outputs)
    return net_ho, logits        


In [ ]:

def generator_txt2img_resnet(input_z, t_txt=None, is_train=True, reuse=False, batch_size=batch_size):
    """ z + (txt) --> 64x64 """
    # https://github.com/hanzhanggit/StackGAN/blob/master/stageI/model.py
    s = 64 # output image size [64]
    s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)
    gf_dim = 128

    w_init = tf.random_normal_initializer(stddev=0.02)
    gamma_init = tf.random_normal_initializer(1., 0.02)

    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
        tl.layers.set_name_reuse(True)
        net_in = InputLayer(input_z, name='g_inputz')
        print(" 1 ",net_in.outputs.shape)

        if t_txt is not None:
            net_txt = InputLayer(t_txt, name='g_input_txt')
            print(" 2 ",net_txt.outputs.shape)
            net_txt = DenseLayer(net_txt, n_units=t_dim,
                act=lambda x: tl.act.lrelu(x, 0.2), W_init=w_init, name='g_reduce_text/dense')
            print(" 3 ",net_txt.outputs.shape)
            net_in = ConcatLayer([net_in, net_txt], concat_dim=1, name='g_concat_z_txt')
            print(" 4 ",net_in.outputs.shape)

        net_h0 = DenseLayer(net_in, gf_dim*8*4*4, act=tf.nn.relu,
                W_init=w_init, b_init=None, name='g_h0/dense')
        print(" 5 ",net_h0.outputs.shape)
        net_h0 = BatchNormLayer(net_h0,  #act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h0/batch_norm')
        net_h0 = ReshapeLayer(net_h0, [-1, 4, 4, gf_dim*8], name='g_h0/reshape')
        print(" 6 ",net_h0.outputs.shape)
        
        
        
        
        
        

#         net = Conv2d(net_h0, gf_dim*2, (1, 1), (1, 1),
#                 padding='VALID', act=None, W_init=w_init, b_init=None, name='g_h1_res/conv2d')
#         net = BatchNormLayer(net, act=tf.nn.relu, is_train=is_train,
#                 gamma_init=gamma_init, name='g_h1_res/batch_norm')
#         print(" 7 ",net.outputs.shape)
        
#         net = Conv2d(net, gf_dim*2, (3, 3), (1, 1),
#                 padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h1_res/conv2d2')
#         net = BatchNormLayer(net, act=tf.nn.relu, is_train=is_train,
#                 gamma_init=gamma_init, name='g_h1_res/batch_norm2')
#         print(" 8 ",net.outputs.shape)
        
        
        
        
        
#         net = Conv2d(net, gf_dim*8, (3, 3), (1, 1),
#                 padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h1_res/conv2d3')
#         net = BatchNormLayer(net, # act=tf.nn.relu,
#                 is_train=is_train, gamma_init=gamma_init, name='g_h1_res/batch_norm3')
#         print(" 9 ",net.outputs.shape)
#         net_h1 = ElementwiseLayer(layer=[net_h0, net], combine_fn=tf.add, name='g_h1_res/add')
#         net_h1.outputs = tf.nn.relu(net_h1.outputs)
#         print(" 10 ",net_h1.outputs.shape)

        # Note: you can also use DeConv2d to replace UpSampling2dLayer and Conv2d
        # net_h2 = DeConv2d(net_h1, gf_dim*4, (4, 4), out_size=(s8, s8), strides=(2, 2),
        #         padding='SAME', batch_size=batch_size, act=None, W_init=w_init, b_init=b_init, name='g_h2/decon2d')
        net_h2 = UpSampling2dLayer(net_h0, size=[8, 8], is_scale=False, method=1,
                align_corners=False, name='g_h2/upsample2d')
        net_h2 = Conv2d(net_h2, 512, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h2/conv2d')
        net_h2 = BatchNormLayer(net_h2, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h2/batch_norm')
        print(" up1  ",net_h2.outputs.shape)
        
        net_h2 = UpSampling2dLayer(net_h2, size=[16, 16], is_scale=False, method=1,
                align_corners=False, name='g_h2/upsample2d')
        net_h2 = Conv2d(net_h2, 256, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h21/conv2d1')
        net_h2 = BatchNormLayer(net_h2, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h21/batch_norm1')
        print(" up2  ",net_h2.outputs.shape)
        
        net_h2 = UpSampling2dLayer(net_h2, size=[32, 32], is_scale=False, method=1,
                align_corners=False, name='g_h2/upsample2d')
        net_h2 = Conv2d(net_h2, 128, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h22/conv2d2')
        net_h2 = BatchNormLayer(net_h2, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h22/batch_norm2')
        print(" up3  ",net_h2.outputs.shape)
        
        net_h2 = UpSampling2dLayer(net_h2, size=[64, 64], is_scale=False, method=1,
                align_corners=False, name='g_h2/upsample2d')
        net_h2 = Conv2d(net_h2,64, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h23/conv2d3')
        net_h2 = BatchNormLayer(net_h2, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h23/batch_norm3')
        print(" up4  ",net_h2.outputs.shape)
        
        
        net_ho = Conv2d(net_h2, c_dim, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, name='g_h4/conv2d4')
        print(" up1  ",net_ho.outputs.shape)
        
        net_ho.outputs = tf.nn.tanh(net_ho.outputs)
        
        
        
    
        net_h3 = Conv2d(net_ho, 128, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, name='g_h6/conv2d6')
      
        net_h3.outputs = tf.nn.relu(net_h3.outputs)
        print(" zero conv  ",net_h3.outputs.shape)
        
        
        net_h3 = Conv2d(net_h3, 256, (4, 4), (2, 2),
                padding='SAME', act=None, W_init=w_init, name='g_h7/conv2d7')
      
        net_h3.outputs = tf.nn.relu(net_h3.outputs)
        print(" zero two  ",net_h3.outputs.shape)
        
        net_h3 = Conv2d(net_h3, 512, (4, 4), (2, 2),
                padding='SAME', act=None, W_init=w_init, name='g_h8/conv2d8')
      
        net_h3.outputs = tf.nn.relu(net_h3.outputs)
        print(" zero three  ",net_h3.outputs.shape)
        
        
        net_E = ExpandDimsLayer(net_h3, 1, name='g_h9/expanddim9')
        net_E = ExpandDimsLayer(net_E, 1, name='g_h9/expanddim10')
        net_E = TileLayer(net_E, [1, 16, 16, 1], name='g_h9/tile9')
        net_E = ConcatLayer([net_E, net_txt], concat_dim=3, name='10/g_h9d_h10')
        print(" zero four  ",net_E.outputs.shape)
        
        
        
        

#         net = Conv2d(net_h2, gf_dim, (1, 1), (1, 1),
#                 padding='VALID', act=None, W_init=w_init, b_init=None, name='g_h3_res/conv2d')
#         net = BatchNormLayer(net, act=tf.nn.relu, is_train=is_train,
#                 gamma_init=gamma_init, name='g_h3_res/batch_norm')
#         net = Conv2d(net, gf_dim, (3, 3), (1, 1),
#                 padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h3_res/conv2d2')
#         net = BatchNormLayer(net, act=tf.nn.relu, is_train=is_train,
#                 gamma_init=gamma_init, name='g_h3_res/batch_norm2')
#         net = Conv2d(net, gf_dim*4, (3, 3), (1, 1),
#                 padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h3_res/conv2d3')
#         net = BatchNormLayer(net, #act=tf.nn.relu,
#                 is_train=is_train, gamma_init=gamma_init, name='g_h3_res/batch_norm3')
#         net_h3 = ElementwiseLayer(layer=[net_h2, net], combine_fn=tf.add, name='g_h3/add')
#         net_h3.outputs = tf.nn.relu(net_h3.outputs)
#         print(" 11 ",net_h3.outputs.shape)
        # net_h4 = DeConv2d(net_h3, gf_dim*2, (4, 4), out_size=(s4, s4), strides=(2, 2),
        #         padding='SAME', batch_size=batch_size, act=None, W_init=w_init, b_init=b_init, name='g_h4/decon2d'),
        net_h4 = UpSampling2dLayer(net_h3, size=[s8, s8], is_scale=False, method=1,
                align_corners=False, name='g_h4/upsample2d')
        net_h4 = Conv2d(net_h4, gf_dim*2, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h4/conv2d')
        net_h4 = BatchNormLayer(net_h4, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h4/batch_norm')
        print(" 12 ",net_h4.outputs.shape)
        
        
        
        #====================================================================================
        
        net_h4 = UpSampling2dLayer(net_h4, size=[s4, s4], is_scale=False, method=1,
                align_corners=False, name='g_h4d/dupsample2d')
        net_h4 = Conv2d(net_h4, gf_dim*2, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h4d/dconv2d')
        net_h4 = BatchNormLayer(net_h4, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h4d/dbatch_norm')
        print(" 20 ",net_h4.outputs.shape)
        
#         net_h4 = UpSampling2dLayer(net_h4, size=[s2, s2], is_scale=False, method=1,
#                 align_corners=False, name='g_h4f/fupsample2d')
#         net_h4 = Conv2d(net_h4, gf_dim*2, (3, 3), (1, 1),
#                 padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h4f/fconv2d')
#         net_h4 = BatchNormLayer(net_h4, act=tf.nn.relu,
#                 is_train=is_train, gamma_init=gamma_init, name='g_h4f/fbatch_norm')
#         print(" 22 ",net_h4.outputs.shape)
        
        
        
        
        
        
        
        #=======================================================================================
        # net_h5 = DeConv2d(net_h4, gf_dim, (4, 4), out_size=(s2, s2), strides=(2, 2),
        #         padding='SAME', batch_size=batch_size, act=None, W_init=w_init, b_init=b_init, name='g_h5/decon2d')
        net_h5 = UpSampling2dLayer(net_h4, size=[s2, s2], is_scale=False, method=1,
                align_corners=False, name='g_h5/upsample2d')
        net_h5 = Conv2d(net_h5, gf_dim, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, b_init=None, name='g_h5/conv2d')
        net_h5 = BatchNormLayer(net_h5, act=tf.nn.relu,
                is_train=is_train, gamma_init=gamma_init, name='g_h5/batch_norm')
        print(" 12 ",net_h5.outputs.shape)
        # net_ho = DeConv2d(net_h5, c_dim, (4, 4), out_size=(s, s), strides=(2, 2),
        #         padding='SAME', batch_size=batch_size, act=None, W_init=w_init, name='g_ho/decon2d')
        net_ho = UpSampling2dLayer(net_h5, size=[s, s], is_scale=False, method=1,
                align_corners=False, name='g_ho/upsample2d')
        net_ho = Conv2d(net_ho, c_dim, (3, 3), (1, 1),
                padding='SAME', act=None, W_init=w_init, name='g_ho/conv2d')
        
        print(" 13 ",net_h5.outputs.shape)
        logits = net_ho.outputs
        net_ho.outputs = tf.nn.tanh(net_ho.outputs)
        print(" outputs  ",net_ho.outputs.shape)
    return net_ho, logits


In [ ]:

t_real_image = tf.placeholder('float32', [batch_size, image_size, image_size, 3], name = 'real_image')
print(t_real_image.shape)
t_wrong_image = tf.placeholder('float32', [batch_size ,image_size, image_size, 3], name = 'wrong_image')
print(t_wrong_image.shape)
t_real_caption = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name='real_caption_input')
print(t_real_caption.shape)
t_wrong_caption = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name='wrong_caption_input')
print(t_wrong_caption.shape)
t_z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z_noise')
print(t_z.shape)

## training inference for text-to-image mapping
net_cnn = cnn_encoder(t_real_image, is_train=True, reuse=False)
x = net_cnn.outputs
v = rnn_embed(t_real_caption, is_train=True, reuse=False).outputs
x_w = cnn_encoder(t_wrong_image, is_train=True, reuse=True).outputs
v_w = rnn_embed(t_wrong_caption, is_train=True, reuse=True).outputs

alpha = 0.2 # margin alpha
rnn_loss = tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x, v_w))) + \
            tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x_w, v)))

## training inference for txt2img
generator_txt2img = generator_txt2img_resnet
discriminator_txt2img = discriminator_txt2img_resnet

net_rnn = rnn_embed(t_real_caption, is_train=False, reuse=True)
net_fake_image, _ = generator_txt2img(t_z,
                net_rnn.outputs,
                is_train=True, reuse=False, batch_size=batch_size)
                #+ tf.random_normal(shape=net_rnn.outputs.get_shape(), mean=0, stddev=0.02), # NOISE ON RNN
net_d, disc_fake_image_logits = discriminator_txt2img(
                net_fake_image.outputs, net_rnn.outputs, is_train=True, reuse=False)
_, disc_real_image_logits = discriminator_txt2img(
                t_real_image, net_rnn.outputs, is_train=True, reuse=True)
_, disc_mismatch_logits = discriminator_txt2img(
                # t_wrong_image,
                t_real_image,
                # net_rnn.outputs,
                rnn_embed(t_wrong_caption, is_train=False, reuse=True).outputs,
                is_train=True, reuse=True)

## testing inference for txt2img
net_g, _ = generator_txt2img(t_z,
                rnn_embed(t_real_caption, is_train=False, reuse=True).outputs,
                is_train=False, reuse=True, batch_size=batch_size)

d_loss1 = tl.cost.sigmoid_cross_entropy(disc_real_image_logits, tf.ones_like(disc_real_image_logits), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(disc_mismatch_logits,  tf.zeros_like(disc_mismatch_logits), name='d2')
d_loss3 = tl.cost.sigmoid_cross_entropy(disc_fake_image_logits, tf.zeros_like(disc_fake_image_logits), name='d3')
d_loss = d_loss1 + (d_loss2 + d_loss3) * 0.5
g_loss = tl.cost.sigmoid_cross_entropy(disc_fake_image_logits, tf.ones_like(disc_fake_image_logits), name='g')

print("Stage 1 sucesss")

In [ ]:
lr = 0.001
lr_decay = 0.5      # decay factor for adam, https://github.com/reedscot/icml2016/blob/master/main_cls_int.lua  https://github.com/reedscot/icml2016/blob/master/scripts/train_flowers.sh
decay_every = 100   # https://github.com/reedscot/icml2016/blob/master/main_cls.lua
beta1 = 0.5

cnn_vars = tl.layers.get_variables_with_name('cnn', True, True)
rnn_vars = tl.layers.get_variables_with_name('rnn', True, True)
d_vars = tl.layers.get_variables_with_name('discriminator', True, True)
g_vars = tl.layers.get_variables_with_name('generator', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr, trainable=False)
    d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars )
    g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars )
    # e_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(e_loss, var_list=e_vars + c_vars)
    grads, _ = tf.clip_by_global_norm(tf.gradients(rnn_loss, rnn_vars + cnn_vars), 10)
    optimizer = tf.train.AdamOptimizer(lr_v, beta1=beta1)# optimizer = tf.train.GradientDescentOptimizer(lre)
    rnn_optim = optimizer.apply_gradients(zip(grads, rnn_vars + cnn_vars))


In [ ]:
    print("devima 1")
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    print("devima 1.1")
    tl.layers.initialize_global_variables(sess)
    ## seed for generation, z and sentence ids
    
    print("devima 2")
    sample_size = batch_size
    sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, z_dim)).astype(np.float32)
        # sample_seed = np.random.uniform(low=-1, high=1, size=(sample_size, z_dim)).astype(np.float32)]
    n = int(sample_size / ni)
    print("devima 3")
    sample_sentence = ["this bird has a bright yellow body, with brown on its crown and wings."] * n + \
                      ["this bird has a red breast and belly as well as a small bill."] * n + \
                      ["small, roundish bird with off white breast and belly, light brown crown, brown and black colored wings."] * n + \
                      ["a white bird with a black crown and yellow beak"] * n + \
                      ["the bird has gray crown, belly and white abdomen, with black tarsus and feet"] * n + \
                      ["a colorful bird with a bright yellow body, a black crown and throat, orange bill, and black primaries and secondaries."] * n + \
                      ["this bird has wings that are blue and white."] * n +\
                      ["these white bird have wings off white in color and end in a white towards the tips."] * n

    # sample_sentence = captions_ids_test[0:sample_size]
    print("devima 3")
    for i, sentence in enumerate(sample_sentence):
        print("seed: %s" % sentence)
        sentence = preprocess_caption(sentence)
        sample_sentence[i] = [vocab[word] for word in nltk.tokenize.word_tokenize(sentence)] + [vocab['</S>']]    # add END_ID
        # sample_sentence[i] = [vocab.word_to_id(word) for word in sentence]
        # print(sample_sentence[i])
    sample_sentence = tl.prepro.pad_sequences(sample_sentence, padding='post')
    print("devima 4")
    n_epoch = 1000
    print_freq = 1
    n_batch_epoch = int(n_images_train / batch_size)
    print("n_batch_epoch",  n_batch_epoch)
    avg_err_d_array =[]
    avg_err_g_array =[]
    avg_err_r_array =[]
    print("devima 5")
    # exit()
    for epoch in range(0, n_epoch+1):
        print("devima 6")
        err_d_array =[]
        err_g_array =[]
        err_r_array =[]
        start_time = time.time()

        if epoch !=0 and (epoch % decay_every == 0):
            new_lr_decay = lr_decay ** (epoch // decay_every)
            sess.run(tf.assign(lr_v, lr * new_lr_decay))
            log = " ** new learning rate: %f" % (lr * new_lr_decay)
            print(log)
            # logging.debug(log)
        elif epoch == 0:
            log = " ** init lr: %f  decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
            print(log)

        for step in range(n_batch_epoch):
            step_time = time.time()
            ## get matched text
            idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
            b_real_caption = captions_ids_train[idexs]
            b_real_caption = tl.prepro.pad_sequences(b_real_caption, padding='post')
            ## get real image
            b_real_images = images_train[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
            # save_images(b_real_images, [ni, ni], 'samples/step1_gan-cls/train_00.png')
            ## get wrong caption
            idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
            b_wrong_caption = captions_ids_train[idexs]
            b_wrong_caption = tl.prepro.pad_sequences(b_wrong_caption, padding='post')
            ## get wrong image
            idexs2 = get_random_int(min=0, max=n_images_train-1, number=batch_size)
            b_wrong_images = images_train[idexs2]
            ## get noise
            b_z = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, z_dim)).astype(np.float32)
                # b_z = np.random.uniform(low=-1, high=1, size=[batch_size, z_dim]).astype(np.float32)

            b_real_images = threading_data(b_real_images, prepro_img, mode='train')   # [0, 255] --> [-1, 1] + augmentation
            b_wrong_images = threading_data(b_wrong_images, prepro_img, mode='train')
            ## updates text-to-image mapping
            if epoch < 50:
                errRNN, _ = sess.run([rnn_loss, rnn_optim], feed_dict={
                                                t_real_image : b_real_images,
                                                t_wrong_image : b_wrong_images,
                                                t_real_caption : b_real_caption,
                                                t_wrong_caption : b_wrong_caption})
            else:
                errRNN = 0

            ## updates D
            errD, _ = sess.run([d_loss, d_optim], feed_dict={
                            t_real_image : b_real_images,
                            # t_wrong_image : b_wrong_images,
                            t_wrong_caption : b_wrong_caption,
                            t_real_caption : b_real_caption,
                            t_z : b_z})
            ## updates G
            errG, _ = sess.run([g_loss, g_optim], feed_dict={
                            t_real_caption : b_real_caption,
                            t_z : b_z})

            print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.8f, g_loss: %.8f, rnn_loss: %.8f" \
                        % (epoch, n_epoch, step, n_batch_epoch, time.time() - step_time, errD, errG, errRNN))
            err_d_array.append(errD)
            err_g_array.append(errG)
            err_r_array.append(errRNN)
            

        if (epoch + 1) % print_freq == 0:
            print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
            img_gen, rnn_out = sess.run([net_g.outputs, net_rnn.outputs], feed_dict={
                                        t_real_caption : sample_sentence,
                                        t_z : sample_seed})

            # img_gen = threading_data(img_gen, prepro_img, mode='rescale')
            save_images(img_gen, [ni, ni], 'samples/step1_gan-cls/train_{:02d}.png'.format(epoch))
            
        avg_err_d_array.append((sum(err_d_array)/ len(err_d_array)))
        avg_err_g_array.append((sum(err_g_array)/len(err_g_array)))
        avg_err_r_array.append((sum(err_g_array)/len(err_g_array)))
 
    with open("_save_error.pickle", 'wb') as f:
        pickle.dump((avg_err_d_array,avg_err_g_array,avg_err_r_array), f)
